In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, precision_recall_curve, f1_score, recall_score
from imblearn.over_sampling import SMOTE


### 1. 데이터 로드 및 기본 탐색 ###

In [28]:
df = pd.read_csv('creditcard.csv')

# 데이터 구조 확인
print("\n[Data Head]")
print(df.head())

print("\n[Data Info]")
print(df.info())

print("\n[Data Describe]")
print(df.describe())

# Class 비율 및 건수 확인
class_counts = df['Class'].value_counts()
print("\n[Class Counts]")
print(class_counts)
print(f"Normal(0): {class_counts[0]}, Fraud(1): {class_counts[1]}")
print(f"Fraud Ratio: {class_counts[1] / len(df):.4f}")


[Data Head]
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V

### 2. 샘플링 ###

In [29]:
# 사기 거래(Class=1)는 전부 유지
fraud_df = df[df['Class'] == 1]

# 정상 거래(Class=0)는 10,000건만 무작위 샘플링 (random_state=42)
normal_df = df[df['Class'] == 0].sample(n=10000, random_state=42)

# 두 데이터셋을 합쳐 새로운 분석용 데이터프레임 생성
new_df = pd.concat([fraud_df, normal_df], axis=0)

# 셔플 (Optional but recommended)
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 샘플링 후 Class 비율 출력
print("\n[Class Counts after Sampling]")
print(new_df['Class'].value_counts())
print(f"Normal(0): {new_df['Class'].value_counts()[0]}, Fraud(1): {new_df['Class'].value_counts()[1]}")


[Class Counts after Sampling]
Class
0    10000
1      492
Name: count, dtype: int64
Normal(0): 10000, Fraud(1): 492


### 3. 데이터 전처리 ###

In [30]:
# Amount 변수 표준화 (StandardScaler)
scaler = StandardScaler()
# reshape(-1, 1) is needed for a single feature
new_df['Amount_Scaled'] = scaler.fit_transform(new_df['Amount'].values.reshape(-1, 1))

# Amount 원본 변수 제거
new_df.drop(['Amount'], axis=1, inplace=True)
# new_df.drop(['Time'], axis=1, inplace=True)

# X, y 분리
X = new_df.drop('Class', axis=1)
y = new_df['Class']

print("Features shape:", X.shape)
print("Target shape:", y.shape)


Features shape: (10492, 30)
Target shape: (10492,)


### 4. 학습 데이터와 테스트 데이터 분할 ###

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Train set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# 분할된 데이터의 Class 비율 출력
print("\n[Train Class Ratio]")
print(y_train.value_counts(normalize=True))
print("\n[Test Class Ratio]")
print(y_test.value_counts(normalize=True))


Train set shape: (8393, 30)
Test set shape: (2099, 30)

[Train Class Ratio]
Class
0    0.953056
1    0.046944
Name: proportion, dtype: float64

[Test Class Ratio]
Class
0    0.953311
1    0.046689
Name: proportion, dtype: float64


### 5. SMOTE 적용 ###

In [32]:
# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# SMOTE 적용 전후 건수 출력
print(f"Before SMOTE - Class 1 counts: {sum(y_train == 1)}")
print(f"After SMOTE  - Class 1 counts: {sum(y_train_res == 1)}")


Before SMOTE - Class 1 counts: 394
After SMOTE  - Class 1 counts: 7999


데이터 불균형(Imbalanced Data) 문제는 머신러닝 모델이 다수 클래스(Normal)에 편향되게 학습하도록 만듭니다.
단순히 소수 클래스(Fraud)를 복제하는 Random Oversampling은 과적합(Overfitting) 위험이 있습니다.
SMOTE(Synthetic Minority Over-sampling Technique)는 소수 클래스 데이터 포인트 사이의 보간(interpolation)을 통해
새로운 가상의 데이터를 생성함으로써 과적합 위험을 줄이면서 데이터 균형을 맞추는 효과적인 방법입니다.

### 6. 모델 학습 ###

In [33]:
# 적합한 ML 모델 선정: Random Forest Classifier
# (복잡한 패턴 인식에 유리하고 튜닝 없이도 성능이 비교적 좋음)
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_res, y_train_res)

# 예측값 및 예측 확률
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] # Probability for Class 1

# Classification Report
print("\n[Classification Report]")
print(classification_report(y_test, y_pred))

# PR-AUC 계산
pr_auc = average_precision_score(y_test, y_pred_proba)
print(f"PR-AUC Score: {pr_auc:.4f}")



[Classification Report]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2001
           1       0.98      0.83      0.90        98

    accuracy                           0.99      2099
   macro avg       0.98      0.91      0.95      2099
weighted avg       0.99      0.99      0.99      2099

PR-AUC Score: 0.9157


### 7. 최종 성능 평가 ###

In [34]:
# 목표 지표
target_recall = 0.80
target_f1 = 0.88
target_prauc = 0.90

# 현재 성능 계산 (Class 1 기준)
current_recall_1 = recall_score(y_test, y_pred, pos_label=1)
current_f1_1 = f1_score(y_test, y_pred, pos_label=1)

# 현재 성능 계산 (Class 0 기준 - 문제에서 'Class 0, 1 둘 다'라고 언급했으므로)
current_recall_0 = recall_score(y_test, y_pred, pos_label=0)
current_f1_0 = f1_score(y_test, y_pred, pos_label=0)

print(f"Target: Recall >= {target_recall}, F1 >= {target_f1}, PR-AUC >= {target_prauc}")
print("-" * 30)
print(f"Class 1 (Fraud) - Recall: {current_recall_1:.4f}, F1: {current_f1_1:.4f}")
print(f"Class 0 (Normal) - Recall: {current_recall_0:.4f}, F1: {current_f1_0:.4f}")
print(f"PR-AUC: {pr_auc:.4f}")
print("-" * 30)

# 달성 여부 판단 로직
# (여기서는 엄격하게 Class 1의 성능을 위주로 판단하거나, 둘 다 만족하는지 체크)
is_pass = (current_recall_1 >= target_recall) and \
          (current_f1_1 >= target_f1) and \
          (pr_auc >= target_prauc)

if is_pass:
    print("결과: 목표 달성 성공! (Goals Achieved)")
else:
    print("결과: 목표 달성 실패 (Goals Not Met)")
    print("\n[추가 제안 방법 (Suggestions)]")
    print("1. 하이퍼파라미터 튜닝: GridSearchCV나 Optuna를 사용하여 max_depth, n_estimators 등을 조정.")
    print("2. Threshold 조정: Precision-Recall Curve를 그려 최적의 임계값(Threshold) 탐색.")
    print("3. 모델 변경: XGBoost, LightGBM, CatBoost 등 부스팅 계열 모델 시도.")
    print("4. 특징 공학(Feature Engineering): Time 변수 변환 또는 V1~V28 변수 조합 시도.")

Target: Recall >= 0.8, F1 >= 0.88, PR-AUC >= 0.9
------------------------------
Class 1 (Fraud) - Recall: 0.8265, F1: 0.8950
Class 0 (Normal) - Recall: 0.9990, F1: 0.9953
PR-AUC: 0.9157
------------------------------
결과: 목표 달성 성공! (Goals Achieved)
